In [14]:
import numpy as np
from iminuit import Minuit
from scipy.integrate import quad

from iminuit import Minuit

import constants, functions

n_panth = 1701
cov_mat = np.array(constants.cov_mat).reshape(n_panth, n_panth)

def dL(z, omega, H0):
    borne = (omega/(1-omega)) ** (1/3)
    X = 1 / (H0 * np.sqrt(1-omega))
    def integrale(z_i):
        res, _ = quad(lambda t: 1/np.sqrt(1 + t**3), borne, borne * (1+z_i))
        return constants.C * (1+z_i) * X * res
    return np.array([integrale(z_i) for z_i in np.atleast_1d(z)])

def mu(z, omega, H0):
    return 5 * np.log10(dL(z, omega, H0)) + 25

def chi2_panth(omega, H0, M):
    delta_mu = np.empty(n_panth)

    for i in range(n_panth):
        if constants.is_calibrator_panth[i] == 0:
            delta_mu[i] = mu(constants.z_data_panth[i],omega, H0) - (constants.m_b_corr_panth[i] - M)
        else:
            delta_mu[i] = (constants.m_b_corr_panth[i] - M) - constants.ceph_dist_panth[i]

    delta_mu = mu(constants.z_data_panth, omega, H0) - constants.mu_data_panth
    return np.dot(np.dot(cov_mat, delta_mu), delta_mu)

def chi2_rsd(omega, sigma, gamma):
    model = functions.growth(constants.z_data, gamma=gamma, omega_0=omega, sigma_8_0=sigma)

    errors = np.asarray(0.5 * (constants.fs8_err_plus + constants.fs8_err_minus))
    residuals = (model - constants.fs8_data.values) / errors
    return np.sum(residuals**2)

def chi2tot(omega, sigma, gamma, H0, M):
    return chi2_rsd(omega, sigma, gamma) + chi2_panth(omega, H0, M)

In [ ]:
n_om = 100
om_min = .05
om_max = 1

n_s8 = 100
s8_min = 0.4
s8_max = 1.2

n_gamma = 100
gamma_min = -0.5
gamma_max = 2

omega_vals = np.asarray(np.linspace(om_min, om_max, n_om))
sigma_vals = np.asarray(np.linspace(s8_min, s8_max, n_s8))
gamma_vals = np.asarray(np.linspace(gamma_min, gamma_max, n_gamma))

minimizer = Minuit(chi2tot, omega=0.315, sigma=0.811, gamma=0.55, M=-19.25, H0 = 73.4)

minimizer.limits["omega"] = (om_min,om_max)
minimizer.limits["sigma"] = (s8_min,s8_max)
minimizer.limits["gamma"] = (gamma_min,gamma_max)

result = minimizer.migrad()

/tmp/ipykernel_22577/2324504931.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  delta_mu[i] = mu(constants.z_data_panth[i],omega, H0) - (constants.m_b_corr_panth[i] - M)
/tmp/ipykernel_22577/2324504931.py:21: RuntimeWarning: invalid value encountered in log10
  return 5 * np.log10(dL(z, omega, H0)) + 25
/tmp/ipykernel_22577/2324504931.py:16: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  res, _ = quad(lambda t: 1/np.sqrt(1 + t**3), borne, borne * (1+z_i))


In [24]:
print(f"\nRésultats du fit :")
print(f"Chi2 = {minimizer.fval:.2f} \n")
print(f"Paramètres estimés :")
print(f"Ωm = {minimizer.values['omega']:.10f} ± {minimizer.errors['omega']:.10f}")
print(f"σ8,0 = {minimizer.values['sigma']:.10f} ± {minimizer.errors['sigma']:.10f}")
print(f"γ = {minimizer.values['gamma']:.10f} ± {minimizer.errors['gamma']:.10f}\n")
print(f"H0 = {minimizer.values['H0']:.10f} ± {minimizer.errors['H0']:.10f}")
print(f"M = {minimizer.values['M']:.10f} ± {minimizer.errors['M']:.10f}")

print(result)


Résultats du fit :
Chi2 = 11373.28 

Paramètres estimés :
Ωm = 0.0500000000 ± 0.0000382471
σ8,0 = 0.5359079619 ± 0.0558023641
γ = 0.0600642048 ± 0.0518551977

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 1.137e+04                  │              Nfcn = 142              │
│ EDM = 1.62e-06 (Goal: 0.0002)    │            time = 9.7 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│     SOME parameters at limit     │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└───